In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, Conv2D, add

def residual_block(x, filters, pooling=False):
    # input pointer
    residual = x
    x = Conv2D(filters, kernel_size=3, activation="relu", padding="same")(x)
    x = Conv2D(filters, kernel_size=3, activation="relu", padding="same")(x)

    if pooling:
        x = MaxPooling2D(2, padding="same")(x)
        # project to expected shape via strides convolution
        residual = Conv2D(filters, kernel_size=1, strides=2)(residual)
    elif filters != residual.shape[-1]:
        # if number of channels has changed
        residual = Conv2D(filters, kernel_size=1)(residual)
    
    x = add([x, residual])
    return x

In [2]:
from tensorflow.keras.layers import Input, Rescaling, Dense, GlobalAveragePooling2D, Flatten

inputs = Input(shape=(28, 28, 1))
x = Rescaling(1. /255)(inputs)

x = residual_block(x, filters=32, pooling=True)
x = residual_block(x, filters=64, pooling=True)
x = residual_block(x, filters=128, pooling=False)

x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=outputs)